In [8]:
'''
Base chess game for 2 human players with no model implemetation
'''

import pygame
import time
import sys
import chess
import chess.svg
from joblib import load
import pandas as pd
import math

#Functions for parsing FEN to ML predict

def convertFenToInput(fenString):
    input = []

    # FEN strings are easier to parse when split up like this
    parts = fenString.split(' ')

    # Encode the board
    for char in parts[0]:
        if char != '/':
            if char.isalpha():
                input.append(convertPieceToInputValue(char, parts[1]))

            if char.isnumeric():
                zeroes = [0] * int(char)
                for val in zeroes:
                    input.append(val)

    # Encode the turn player
    input.append(1 if parts[1] == 'w' else -1)

    # Encode castling availability
    castling = [0] * 4
    for char in parts[2]:
        if char == 'K':
            castling[0] = 1
        if char == 'Q':
            castling[1] = 1
        if char == 'k':
            castling[2] = 1
        if char == 'q':
            castling[3] = 1
    for val in castling:
        input.append(val)
    #input.append(castling)

    return input


def convertPieceToInputValue(char, player):
    pieceDict = {
    'K': 1.0,
    'Q': 0.8,
    'B': 0.6,
    'N': 0.5,
    'R': 0.4,
    'P': 0.2,
    'k': -1.0,
    'q': -0.8,
    'b': -0.6,
    'n': -0.5,
    'r': -0.4,
    'p': -0.2
    }

    value = pieceDict[char] if char in pieceDict else 0
    return value

In [9]:
starting_order = {}

for i in range(8):
    for j in range(8):
        starting_order[i, j] = None

#rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1

def board_from_fen(fen):
    i = 0
    j = 0
    for char in fen:
        if char == '/':
            j += 1
            i = 0
        elif char.isnumeric():
            spaces = int(char) #char == '8'
            for pivot in range(spaces):
                starting_order[i, j] = None
                i += 1
        elif char.isalpha():
            starting_order[i, j] = char_to_pygame_image(char)
            i += 1
        else:
            break

def char_to_pygame_image(char):
    if char == 'p':
        return pygame.image.load('src/img/b_pawn.png')
    elif char == 'r':
        return pygame.image.load('src/img/b_rook.png')
    elif char == 'n':
        return pygame.image.load('src/img/b_knight.png')
    elif char == 'b':
        return pygame.image.load('src/img/b_bishop.png')
    elif char == 'q':
        return pygame.image.load('src/img/b_queen.png')
    elif char == 'k':
        return pygame.image.load('src/img/b_king.png')
    elif char == 'P':
        return pygame.image.load('src/img/w_pawn.png')
    elif char == 'R':
        return pygame.image.load('src/img/w_rook.png')
    elif char == 'N':
        return pygame.image.load('src/img/w_knight.png')
    elif char == 'B':
        return pygame.image.load('src/img/w_bishop.png')
    elif char == 'Q':
        return pygame.image.load('src/img/w_queen.png')
    elif char == 'K':
        return pygame.image.load('src/img/w_king.png')

board = chess.Board()

fen_start = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'
board = chess.Board(fen_start)
board_from_fen(fen_start)

## returns the input if the input is within the boundaries of the board
def on_board(position):
    if position[0] > -1 and position[1] > -1 and position[0] < 8 and position[1] < 8:
        return True

WIDTH = 504

WIN = pygame.display.set_mode((WIDTH, WIDTH))

""" 
This is creating the window that we are playing on, it takes a tuple argument which is the dimensions of the window so in this case 800 x 800px
"""

pygame.display.set_caption("Chiggity Chess")
WHITE = (240, 216, 191)
GREY = (186, 85, 70)
BLUE = (220, 172, 230)
BLACK = (0, 0, 0)

class Node:
    def __init__(self, row, col, width):
        self.row = row
        self.col = col
        self.x = int(row * width)
        self.y = int(col * width)
        self.colour = WHITE
        self.occupied = None

    def draw(self, WIN):
        pygame.draw.rect(WIN, self.colour, (self.x, self.y, WIDTH / 8, WIDTH / 8))

    def setup(self, WIN):
        if starting_order[(self.row, self.col)]:
            if starting_order[(self.row, self.col)] == None:
                pass
            else:
                WIN.blit(starting_order[(self.row, self.col)], (self.x, self.y))
        """
        For now it is drawing a rectangle but eventually we are going to need it
        to use blit to draw the chess pieces instead
        """

def make_grid(rows, width):
    grid = []
    gap = WIDTH // rows
    print(gap)
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            node = Node(j, i, gap)
            grid[i].append(node)
            if (i + j) % 2 == 1:
                grid[i][j].colour = GREY
    return grid
"""
This is creating the nodes thats are on the board(so the chess tiles)
I've put them into a 2d array which is identical to the dimesions of the chessboard
"""

def draw_grid(win, rows, width):
    gap = width // 8
    for i in range(rows):
        pygame.draw.line(win, BLACK, (0, i * gap), (width, i * gap))
        for j in range(rows):
            pygame.draw.line(win, BLACK, (j * gap, 0), (j * gap, width))
    """
    The nodes are all white so this we need to draw the grey lines that separate all the chess tiles
    from each other and that is what this function does
    """

def update_display(win, grid, rows, width):
    for row in grid:
        for spot in row:
            spot.draw(win)
            spot.setup(win)
    draw_grid(win, rows, width)
    pygame.display.update()

def Find_Node(pos, WIDTH):
    interval = WIDTH / 8
    y, x = pos
    rows = y // interval
    columns = x // interval
    return int(rows), int(columns)


"""
This takes in 2 co-ordinate parameters which you can get as the position of the piece and then the position of the node it is moving to
you can get those co-ordinates using my old function for swap
"""

def selected_square(x, y):
    y = 8 - y
    y = str(y)
    alpha_x = ""

    alpha_x = str(chr(x+97))
    AN = alpha_x + y

    return AN

def valid_selection(selection,count,grid):
    highlights = []
    identifier = "'" + selection

    if identifier in str(list(board.legal_moves)):
        print("Valid Selection")
        selection1 = selection

        for i in range(len(list(board.legal_moves))):
            check = str(list(board.legal_moves)[i])[0:2]
            if selection1 == check:
                highlights.append(str(list(board.legal_moves)[i])[2:4])
        print(highlights)


        for j in range(len(highlights)):
            ypos = int(highlights[j][1]) - 1

            if highlights[j][0] == "a":
                xpos = 0
            elif highlights[j][0] == "b":
                xpos = 1
            elif highlights[j][0] == "c":
                xpos = 2
            elif highlights[j][0] == "d":
                xpos = 3
            elif highlights[j][0] == "e":
                xpos = 4
            elif highlights[j][0] == "f":
                xpos = 5
            elif highlights[j][0] == "g":
                xpos = 6
            elif highlights[j][0] == "h":
                xpos = 7

            grid[7-ypos][xpos].colour = BLUE  

        print(ypos)
            

    else:
        print("Invalid Selection")
        count = 0
    return selection, count, grid

def remove_highlight(grid):
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if (i+j)%2 == 0:
                grid[i][j].colour = WHITE
            else:
                grid[i][j].colour = GREY
    return grid

def evaluateFEN(fen,model):
    fen = convertFenToInput(fen)
    fen = pd.DataFrame(fen).T
    eval = model.predict(fen)
    
    return eval


def main(WIN, WIDTH):
    selected = False
    select_move = ''
    moves = 0
    piece_to_move=[]
    grid = make_grid(8, WIDTH)
    count = 0
    
    print(str(list(board.legal_moves)))
    while True:
        pygame.time.delay(50) ##stops cpu dying
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            """
            This quits the program if the player closes the window
            """

            if event.type == pygame.MOUSEBUTTONDOWN:
                count = count + 1
                pos = pygame.mouse.get_pos()
                x, y = Find_Node(pos, WIDTH)
                selection = selected_square(x, y)
                print("Alpha-Numeric: ", selection)
                selected = False
                if (count % 2) == 0: 
                    selection2 = selection
                    if selection2 == selection1:
                        count = 0
                        print("Invalid Selection")
                        remove_highlight(grid)
                else:
                    selection1, count, grid = valid_selection(selection,count,grid)
                
                if count == 2:
                    select_move = selection1 + selection2
                print(select_move)
                if len(select_move) == 4:
                    if chess.Move.from_uci(select_move) in board.legal_moves:
                        board.push_uci(select_move)
                        print("Move Sucessful")
                        remove_highlight(grid)
                        select_move = ''
                        print(board.fen())
                        fen = board.fen()
                        board_from_fen(fen)
                        count = 0
                    else:
                        select_move = ''
                        print("Move Failed")
                        remove_highlight(grid)
                        count = 0

            update_display(WIN, grid, 8, WIDTH)

main(WIN, WIDTH)

63
[Move.from_uci('g1h3'), Move.from_uci('g1f3'), Move.from_uci('b1c3'), Move.from_uci('b1a3'), Move.from_uci('h2h3'), Move.from_uci('g2g3'), Move.from_uci('f2f3'), Move.from_uci('e2e3'), Move.from_uci('d2d3'), Move.from_uci('c2c3'), Move.from_uci('b2b3'), Move.from_uci('a2a3'), Move.from_uci('h2h4'), Move.from_uci('g2g4'), Move.from_uci('f2f4'), Move.from_uci('e2e4'), Move.from_uci('d2d4'), Move.from_uci('c2c4'), Move.from_uci('b2b4'), Move.from_uci('a2a4')]
Alpha-Numeric:  e2
Valid Selection
['e3', 'e4']
3

Alpha-Numeric:  e4
e2e4
Move Sucessful
rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1
Alpha-Numeric:  e7
Valid Selection
['e6', 'e5']
4

Alpha-Numeric:  e5
e7e5
Move Sucessful
rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Alpha-Numeric:  b1
Valid Selection
['c3', 'a3']
2

Alpha-Numeric:  c3
b1c3
Move Sucessful
rnbqkbnr/pppp1ppp/8/4p3/4P3/2N5/PPPP1PPP/R1BQKBNR b KQkq - 1 2
Alpha-Numeric:  f8
Valid Selection
['e7', 'd6', 'c5', 'b4', 'a3']
2

Alpha-Numeric:

SystemExit: 